### Load MSFT 1Y history

In [2]:
import mysql.connector
import pandas as pd
import numpy as np

def load_stockdata(symbol):
    serverName = "localhost"
    cnx = mysql.connector.connect(user='michael', password='Password2017',
                                  host=serverName, database='stock_market')
    cursor = cnx.cursor()
    
    query = "select * from (SELECT effective_date, `close`, `adj_close` FROM `market.stock_price` WHERE symbol = '{0}' order\
            by effective_date DESC limit 1000) sub order by effective_date ASC".format(symbol)
    
    cursor.execute(query)
    dfsql = pd.DataFrame(cursor.fetchall(), dtype=np.float)
    dfsql.columns = cursor.column_names
    dfsql.set_index('effective_date', inplace=True)
    return dfsql

In [3]:
df = load_stockdata('MSFT')
df.head()

,close,adj_close
effective_date,,
2014-02-25,37.54,34.129187
2014-02-26,37.47,34.065547
2014-02-27,37.86,34.420112
2014-02-28,38.31,34.829226
2014-03-03,37.78,34.347381


### Use rolling window function to caculate mean / deviation

In [4]:
roll = df.rolling(20)
roll.sum().dropna().head()

,close,adj_close
effective_date,,
2014-03-24,769.09,699.211945
2014-03-25,771.89,701.757542
2014-03-26,774.21,703.866751
2014-03-27,775.71,705.230464
2014-03-28,777.70,707.039657


### Append rolling columns

In [5]:
rolling = df["adj_close"].rolling(20)
df2 = pd.DataFrame()
df2['rolling'] = rolling.sum()
df2['sma20'] = rolling.mean()
df2['stdev'] = rolling.std(ddof=0)
df2['bollinger_bands_upper'] = df2['sma20'] + df2['stdev']
df2['bollinger_bands_lower'] = df2['sma20'] - df2['stdev']

df2.where((pd.notnull(df2)),None)
df2.head(30)


,rolling,sma20,stdev,bollinger_bands_upper,bollinger_bands_lower
effective_date,,,,,
2014-02-25,NaN,NaN,NaN,NaN,NaN
2014-02-26,NaN,NaN,NaN,NaN,NaN
2014-02-27,NaN,NaN,NaN,NaN,NaN
2014-02-28,NaN,NaN,NaN,NaN,NaN
2014-03-03,NaN,NaN,NaN,NaN,NaN
2014-03-04,NaN,NaN,NaN,NaN,NaN
2014-03-05,NaN,NaN,NaN,NaN,NaN
2014-03-06,NaN,NaN,NaN,NaN,NaN
2014-03-07,NaN,NaN,NaN,NaN,NaN


In [6]:
idcdic = df2.to_dict(orient='index')

# insert from the list
for date, row in idcdic.items():
    print(date, row)
    break


2014-02-25 {'rolling': nan, 'sma20': nan, 'stdev': nan, 'bollinger_bands_upper': nan, 'bollinger_bands_lower': nan}


## EWMA & ATR

In [18]:
# get the first SMA for the rolling windows
period = 20
rolling = df[:period]["adj_close"].rolling(period)
#rolling.mean()

# combine the first SMA with the rest close adj price
con = pd.concat([rolling.mean(), df[period:]['adj_close']])
#con.head(30)

dfewma = pd.DataFrame()
dfewma['adj_close'] = df['adj_close']
dfewma['concat'] = con
dfewma['ewma20'] = con.ewm(span=period, adjust=False).mean()
#dfewma['stdev'] = rolling.std(ddof=0)
#dfewma['bollinger_bands_upper'] = df2['sma20'] + df2['stdev']
#dfewma['bollinger_bands_lower'] = df2['sma20'] - df2['stdev']

dfewma = dfewma.where((pd.notnull(dfewma)),None)
dfewma.head(30)


,adj_close,concat,ewma20
effective_date,,,
2014-02-25,34.1292,None,None
2014-02-26,34.0655,None,None
2014-02-27,34.4201,None,None
2014-02-28,34.8292,None,None
2014-03-03,34.3474,None,None
2014-03-04,34.9201,None,None
2014-03-05,34.6474,None,None
2014-03-06,34.6838,None,None
2014-03-07,34.4565,None,None


# see another notebook for ATR